<a href="https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Sed_1Lps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bhalunka Sedimentation Tank Design
Created by William Pennock on 7 August 2020 for the Bhalunka 1 L/s plant in collaboration with AguaClara Reach and Gram Vikas. The calculations were made with reference to:

- The [textbook](https://aguaclara.github.io/Textbook/Sedimentation/Sed_Intro.html)
- The design Mathcad worksheet
- The [CEE 4520 slides](https://github.com/monroews/CEE4520Lectures)
- An incomplete version of [AIDE](https://github.com/AguaClara/aguaclara/blob/master/aguaclara/design/sed_tank.py)

In [15]:
!pip install aguaclara

## Imports

In [16]:
import aguaclara as ac
from aguaclara.core.units import unit_registry as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## General Parameters

In [17]:
# Plant
FlowPlant = 1*(u.L/u.s) # Design flow rate
VolStorage = 6000*u.L # Nominal storage of water tower
TimeRunPlant = (VolStorage/FlowPlant).to(u.hr) # Single run time to fill water tower.
print('The time for a single plant run is ',np.floor(TimeRunPlant.magnitude),' hour and ',np.round((TimeRunPlant.magnitude%1)*60,1),' minutes.',sep='')

# Physical
Temperature = 15*u.degC # Average low temperature for Bhubaneswar
nu = ac.viscosity_kinematic_water(Temperature) # Kinematic viscosity of water
rho = ac.density_water(Temperature) #  Density of water

The time for a single plant run is 1.0 hour and 40.0 minutes.


## Sedimentation Tank Design
 This is a typical masonry sedimentation tank, but with a few key differences:
1. To eliminate the "lost triangle" under the last plate settler and keep the average velocity through the plate settlers closer to the upflow velocity, an inner wall will be built within the sedimentation tank at the location of the bottom of the last plate settler. This will direct flow up at a higher rate and will serve as the weir for the floc hopper, with the floc hopper occupying the space between the inner wall (weir) and the outer wall. Originally, a 60° sloped wall was proposed, but Santiago at APP thought this would be less practical.
2. Having one sedimentation tank removes the necessity of the four-channel system. In its place, there will be an influent pipe from the pipe flocculator to the inlet manifold and a single pipe from outlet manifold to disinfection.
3. To make it possible to drain poorly flocculated water before it reaches the sedimentation tank, it is important to have a tee with a ball valve on the inlet pipe.
4. An upflow velocity of 0.85 mm/s (as opposed to the standard 1 mm/s) was selected as a conservative design in the absence of knowledge of the composition of the raw water. High organic content water could have low density flocs that will have a lower settling velocity that requires a floc blanket with a correspondingly lower upflow velocity.

**Questions:**
- How is the sed tank water level controlled in the plantitas (i.e., without the control weir)?
- How should the drain for the floc hopper be sized?

### General Parameters
Primarily using typical AguaClara conventions, as noted in the code. The upflow velocity is lower to account for the likely presence of natural organic matter in flocs.

We need to know how wide the plate material is. Like is standard 42".

In [18]:
VelUp = 0.85*u.mm/u.s # Upflow velocity of sedimentation tank
AngleSlope = 50*u.deg # Angle of sloped walls at bottom of sedimentation tank

WidthSed = 42 * u.inch # Width of sedimentation tank (this is standard)
LengthSed = (FlowPlant/(VelUp*WidthSed)).to(u.m) # Length of sedimentation tank
LengthSed.to(u.m)
print('The minimum length of the sed tank is',LengthSed)
VelUpSed = FlowPlant/(LengthSed*WidthSed)

DepthFlocBlanket = 1*u.m # ? This sets the height of the sedimentation tank.
## In addition to providing sufficient collisions, this also needs to be deep enough to make flow uniform into the plate settlers.

The minimum length of the sed tank is 1.103 meter


### Floc Characteristics
This is open for debate. For design purposes, I am still running with assumptions of kaolin flocs with perhaps some adsorption of DOM. We didn't analyze the composition of the water, so we only know it can get up to around 10 NTU, and this turbidity was presumably both organic and inorganic.

In [19]:
ShearFlocMax = 0.5*u.Pa # The maximum allowable shear for flocs, based on the textbook derivation from Garland

### Inlet Diffuser Design

APP has demonstrated at Manzaragua that the sed tank performs better if the diffusers aren't flattened at all! Thus the diffusers are simple round pipes.

In [20]:
# Inputs
WidthDiffuser = 1/8*u.inch # Mold fabricated with 1/8" plate steel
HeightDiffuser = 15*u.cm # The diffusers are 15 cm long pieces of pipe
HeadLossDiffuser = 1*u.cm # Standard AguaClara design assumption
DiamNomDiffuser = 1*u.inch # Standard nominal diameter of diffusers
PiStretchDiffuser = 1.2 # Assumed 20% stretch when creating flare
# SpaceDiffuser = 6*u.cm # B_Diffuser (was 6 cm in plantita)

# Calculations

## Maximum Diffuser Velocity
# By minor loss equation
VelMaxDiffuser = (np.sqrt(2*u.g_0*HeadLossDiffuser)).to(u.mm/u.s)
# Calculated based on minor loss equation with K = 1.
print('The maximum velocity of the sedimentation tank diffusers by head loss is ',VelMaxDiffuser)
# By estimated maximum shear on flocs
PiPlaneJet = 0.0124 # Taken from textbook
VelMaxDiffuserShear = ((ShearFlocMax/rho)**(1/2)*(VelUp*WidthSed/(nu*PiPlaneJet))**(1/4)).to(u.mm/u.s) # Eq. 434
print('The maximum velocity of the sedimentation tank diffusers by fluid shear is ',VelMaxDiffuserShear,'.',sep='')
# Diffuser Velocity
VelDiffuser = (VelUp*WidthSed/WidthDiffuser).to(u.mm/u.s) # This jet velocity is lower than the head loss maximum and the shear maximum, so this appears to be adequate.
WidthJetReversed = WidthSed*VelUp/VelDiffuser
print('The sedimentation diffuser velocity is ',VelDiffuser,'.',sep='')
EDRDiffuser = (PiPlaneJet*(VelDiffuser**3/WidthJetReversed)).to(u.mW/u.kg)
HeadLossDiffuser = (VelDiffuser**2/(2*u.g_0)).to(u.cm)
print('The head loss across the diffusers is ',HeadLossDiffuser,'.',sep='')

## Diffuser Flare Dimensions
LocPipeDiffuser = (np.abs(np.array(ac.pipedb['NDinch'])-DiamNomDiffuser.magnitude)).argmin() # Index for diffuser in pipe_database.csv
AreaPVCDiffuser = (np.pi/4)*((ac.pipedb.iloc[LocPipeDiffuser,1]*u.inch)**2-(ac.pipedb.iloc[LocPipeDiffuser,6]*u.inch)**2) # Cross sectional area of 1" PVC pipe (Schedule 40)

ThickStretchDiffuser = ac.pipedb.iloc[LocPipeDiffuser,5]*u.inch/PiStretchDiffuser # Stretched wall thickness of diffuser
LengthOutDiffuser = (AreaPVCDiffuser/2/ThickStretchDiffuser - WidthDiffuser).to(u.cm)# Diffuser Outer Length, B_Diffuser
LengthInDiffuser = (LengthOutDiffuser - (2*ThickStretchDiffuser)).to(u.cm) # Diffuser inner length, S_Diffuser
print('The outer length of the diffuser is ',LengthOutDiffuser, ', and the inner length is ',LengthInDiffuser,'.',sep='')

## Number of Diffusers
NumDiffuser = np.floor((LengthSed/LengthOutDiffuser).to(u.dimensionless)) # Assumes that diffusers will be touching each other.
print('The number of diffusers is ',NumDiffuser.magnitude,'.',sep='')

The maximum velocity of the sedimentation tank diffusers by head loss is  442.9 millimeter / second
The maximum velocity of the sedimentation tank diffusers by fluid shear is 356.3 millimeter / second.
The sedimentation diffuser velocity is 285.6 millimeter / second.
The head loss across the diffusers is 0.4159 centimeter.
The outer length of the diffuser is 5.342 centimeter, and the inner length is 4.779 centimeter.
The number of diffusers is 20.0.


The inlet diffusers will be 15 cm long segments of 1" pipe molded such that they have a flare that is about 5.3 cm long (outer length). These will be placed close together to approximate a continuous line jet, requiring 22 diffusers. They have a sufficiently low velocity to not exceed the design head loss or the maximum shear on the flocs. They have a head loss of about 3.5 cm.

### Inlet Manifold Design

In [21]:
PiFlowManifold = 0.8
VelMaxManifold = (VelDiffuser*np.sqrt(2*(1-PiFlowManifold**2)/(PiFlowManifold**2+1))).to(u.mm/u.s) # Eq. 435
DiamMinManifold = ac.diam_circle(FlowPlant/VelMaxManifold).to(u.inch)
SDR = 26
DiamNomManifold = ac.ND_SDR_available(DiamMinManifold, SDR) 

print('The nominal diameter of the inlet manifold is ',DiamNomManifold,' inches.',sep='')

HeightReverser2Diffuser = 3*u.cm

The nominal diameter of the inlet manifold is 3 inch inches.


/usr/local/lib/python3.6/dist-packages/aguaclara/core/pipes.py:144: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for i in range(len(np.array(ID_SDR_all_available(SDR)))):
/usr/local/lib/python3.6/dist-packages/aguaclara/core/pipes.py:145: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  if np.array(ID_SDR_all_available(SDR))[i] >= (ID.to(u.inch)).magnitude:


The calculated inlet manifold diameter is 3" with a length equal to the length of the sedimentation tank bay (1.1 m). The jet reverser nominal diameter is assumed to be 3" with a length of 1.1 m.

### Plate Settler Design

The plate settlers need to occupy less than full length of the sed tank to account for the lost triangle due to the angle of the plates. 

The sed tank might have a floc hopper at one end and if that is the case then the floc hopper might fill the space of the triangle created by the plate settlers. Alternatively, if a simple 4" PVC pipe is used as the floc hopper, then the sed tank can be shorter and the plate settlers will need to be slightly longer.

Assume 60 cm long plates and then calculate the sed tank length occupied by the plates and verify that there is enough space. Then when building the plant fill the available space with plate settlers which will be more than required.

The following design assumes we will use a pipe for the floc hopper. The pipe floc hopper works and given that the turbidity is low it won't fill that fast.

In [22]:
AnglePlate = 60*u.deg
SpacePlate = 2.5*u.cm
VelCapture = 0.12*u.mm/u.s
ThickPlate = 1*u.mm # This is an assumption of policarbonate sheet thickness

LengthMinPlate = ((SpacePlate*(VelUp/VelCapture-1)+ThickPlate*(VelUp/VelCapture))/(np.sin(AnglePlate)*np.cos(AnglePlate))).to(u.cm)
LengthPlate = 60 * u.cm
print('The length of the plate settlers is ',LengthPlate,sep='')

HeightPlate = np.sin(AnglePlate)*LengthPlate
print('The height of the plates in the sedimentation tank is ',np.round(HeightPlate,1),'.',sep='')
OverlapHorizontalPlate = np.cos(AnglePlate)*LengthPlate
print('The horizontal overlap of the plates is ',OverlapHorizontalPlate,'.',sep='')

SpaceHorizontalPlate = ((ThickPlate + SpacePlate)/np.sin(AnglePlate)).to(u.cm)
NumPlate = np.floor(((LengthSed -OverlapHorizontalPlate) /SpaceHorizontalPlate).to(u.dimensionless))
print('The number of sedimentation plates required is ',NumPlate,'.',sep='')
Lfilledwithplates = NumPlate * SpaceHorizontalPlate + OverlapHorizontalPlate

print('The length of the sed tank that is filled with plates is',Lfilledwithplates)
# check the capture velocity


The length of the plate settlers is 60 centimeter
The height of the plates in the sedimentation tank is 52 centimeter.
The horizontal overlap of the plates is 30 centimeter.
The number of sedimentation plates required is 26 dimensionless.
The length of the sed tank that is filled with plates is 108.1 centimeter


The above design shows that the plates almost exactly fill the length of the sed tank and thus thus chosen length of the plate settlers is reasonable.

The plate settlers will be spaced 2.5 cm apart at a 60° angle, resulting in 26 plates 0.60 m in length. Within the sedimentation tank, these will occupy a height of about 52 cm. The plate modules will be assembled with 1/2" PVC with 3/4" spacers and will be supported with 2" PVC that runs the length of the sed tank in the same direction as the jet reverser. The plates overlap a horizontal distance of 30 cm, and this lost triangle is accounted for in the design.)

### Effluent Manifold Design

We can use the design algorithms in Feature Script. The result likely will be a effluent manifold that is the same diameter at the inlet manifold